In [1]:
from configs import merged_cycle_data_file
import pandas as pd
from sklearn.model_selection import train_test_split

### Load the weather & holiday data

In [29]:
from configs import weather_data_csv

holidays = 'holidays.csv'
hol_df = pd.read_csv(holidays, index_col='date', parse_dates=['date'])
weather_df = pd.read_csv(weather_data_csv, index_col='timestamp', parse_dates=['timestamp'])
weather_df.head()

,temperature,feelsLike,wind,weatherCode
timestamp,,,,
2015-01-01 00:00:00,3,-2,21,116
2015-01-01 01:00:00,3,-2,21,116
2015-01-01 02:00:00,3,-1,20,116
2015-01-01 03:00:00,3,-1,20,116
2015-01-01 04:00:00,4,0,20,116


In [53]:
# get a set of the holiday dates
hol_set= set(hol_df.index.map(lambda x: x.date()))

### Load the merged csv file by reading it in chunks

In [87]:
%%time
from datetime import datetime
import time
chunk_size = 1000000
date_mapper = lambda x: pd.to_datetime(datetime(year=x.year, month=x.month, day=x.day, hour=x.hour))

bike_share_df = pd.DataFrame()
iter_ = pd.read_csv(merged_cycle_data_file, chunksize=chunk_size, iterator=True,
        index_col='Rental Id',
        parse_dates=['End Date', 'Start Date'])

r_start = time.time()
for i, df in enumerate(iter_):
    r_end = time.time()
    print(f'{i+1}. Read rows {chunk_size*i}:{chunk_size*(i+1)} in {r_end-r.start:.3f}. ', end='')

    start = time.time()
    old_size = len(df)
    df = df.dropna()
    # print(f'Old size: {old_size}; New size: {len(df)}')
    # leave only entries that are shorter than 2 hours and have valid duration
#     df = df[(df['Duration'] >= 0) & (df['Duration'] <= 2 * 60 * 60)]
    df = df[df['Duration'] >= 0]
    
    diff = df['End Date'] - df['Start Date'] # compute the difference between the objects
    seconds = diff.map(lambda x: x.total_seconds()) # map to seconds
    df = df[(df['Duration'] == seconds) & (seconds >= 0)] # check if duration matches the result and if the result is positive
    
    
    # keep only year, month, day, hour information from the start date
    df['Start Date'] = df['Start Date'].map(date_mapper)
    
    share_df = df.groupby('Start Date').agg({'Start Date': 'count'}).rename(columns={'Start Date': 'share_count'})
    share_df = share_df.join(weather_df)
    share_df = share_df.reset_index()
    share_df = share_df.dropna()
    
    share_df['month'] = share_df['Start Date'].apply(lambda t: t.month)
    share_df['weekday'] = share_df['Start Date'].apply(lambda t: t.weekday())
    share_df['hour'] = share_df['Start Date'].apply(lambda t: t.hour)
    share_df['is_holiday'] = share_df['Start Date'].map(lambda x: x.date() in hol_set).map(lambda x: '1' if x else '0')
    # check if start date hits on a weekend
    # monday is 0, sunday is 6
    share_df['is_weekend'] = share_df['Start Date'].map(lambda x: x.weekday() > 4).map(lambda x: '1' if x else '0')
    share_df['weatherCode'] = share_df['weatherCode'].map(lambda x: str(int(x)))
    
    bike_share_df = bike_share_df.append(share_df)
    end = time.time()
    print(f'Completed cleaning & merging in {end-start:3.3f} seconds.')
    r_start = time.time()

print('Finished reading!')
bike_share_df = bike_share_df.reset_index().drop(columns=['index']) # fix the index

1. Reading rows 0:1000000. Completed cleaning & merging in 97.826 seconds.
2. Reading rows 1000000:2000000. Completed cleaning & merging in 100.976 seconds.
3. Reading rows 2000000:3000000. Completed cleaning & merging in 95.744 seconds.
4. Reading rows 3000000:4000000. Completed cleaning & merging in 94.829 seconds.
5. Reading rows 4000000:5000000. Completed cleaning & merging in 96.906 seconds.
6. Reading rows 5000000:6000000. Completed cleaning & merging in 95.140 seconds.
7. Reading rows 6000000:7000000. Completed cleaning & merging in 95.473 seconds.
8. Reading rows 7000000:8000000. Completed cleaning & merging in 103.139 seconds.
9. Reading rows 8000000:9000000. Completed cleaning & merging in 104.653 seconds.
10. Reading rows 9000000:10000000. Completed cleaning & merging in 95.514 seconds.
11. Reading rows 10000000:11000000. Completed cleaning & merging in 97.521 seconds.
12. Reading rows 11000000:12000000. Completed cleaning & merging in 103.394 seconds.
13. Reading rows 12000

In [88]:
bike_share_df.shape

(42768, 11)

In [89]:
bike_share_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42768 entries, 0 to 42767
Data columns (total 11 columns):
Start Date     42768 non-null datetime64[ns]
share_count    42768 non-null int64
temperature    42768 non-null float64
feelsLike      42768 non-null float64
wind           42768 non-null float64
weatherCode    42768 non-null object
month          42768 non-null int64
weekday        42768 non-null int64
hour           42768 non-null int64
is_holiday     42768 non-null object
is_weekend     42768 non-null object
dtypes: datetime64[ns](1), float64(3), int64(4), object(3)
memory usage: 3.6+ MB


In [90]:
bike_share_df.head(20)

,Start Date,share_count,temperature,feelsLike,wind,weatherCode,month,weekday,hour,is_holiday,is_weekend
0,2016-01-02 00:00:00,129,8.0,4.0,36.0,122,1,5,0,0,1
1,2016-01-02 01:00:00,61,8.0,4.0,35.0,122,1,5,1,0,1
2,2016-01-02 02:00:00,35,9.0,5.0,33.0,122,1,5,2,0,1
3,2016-01-02 03:00:00,31,9.0,5.0,32.0,122,1,5,3,0,1
4,2016-01-02 04:00:00,33,9.0,5.0,31.0,122,1,5,4,0,1
5,2016-01-02 05:00:00,95,9.0,6.0,30.0,122,1,5,5,0,1
6,2016-01-02 06:00:00,512,10.0,6.0,29.0,122,1,5,6,0,1
7,2016-01-02 07:00:00,1813,10.0,6.0,31.0,122,1,5,7,0,1
8,2016-01-02 08:00:00,3715,10.0,6.0,33.0,122,1,5,8,0,1
9,2016-01-02 09:00:00,1862,10.0,6.0,35.0,122,1,5,9,0,1


In [91]:
bike_share_df.to_csv('shares-1.csv')

In [92]:
bike_share_df[bike_share_df['Start Date'] == '2018-12-07 10:00:00']

,Start Date,share_count,temperature,feelsLike,wind,weatherCode,month,weekday,hour,is_holiday,is_weekend
8115,2018-12-07 10:00:00,996,12.0,10.0,27.0,296,12,4,10,0,0
8959,2018-12-07 10:00:00,292,12.0,10.0,27.0,296,12,4,10,0,0
